In [1]:
!pip install streamlit -q

     |████████████████████████████████| 8.3 MB 14.6 MB/s 
     |████████████████████████████████| 111 kB 56.1 MB/s 
     |████████████████████████████████| 180 kB 61.3 MB/s 
     |████████████████████████████████| 76 kB 4.9 MB/s 
     |████████████████████████████████| 4.3 MB 57.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 124 kB 65.8 MB/s 
     |████████████████████████████████| 788 kB 59.1 MB/s 
     |████████████████████████████████| 374 kB 51.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.21 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.2 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatib

In [23]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
st.title("Sentiment Analysis of Tweets About US Airlines")
st.sidebar.title("Sentiment Analysis of Tweets About US Airlines")
st.markdown("Sentiment Analysis Application✈️ ")
st.sidebar.markdown("Click sidebar elemnts to view ")
data_url = ("https://raw.githubusercontent.com/anjaliasha123/Data_Visualization_projects/master/Streamlit_python/Tweets.csv")
@st.cache
def load_data():
    data = pd.read_csv(data_url)
    data['tweet_created'] = pd.to_datetime(data['tweet_created'])
    return data
data = load_data()
#to display the Tweets
#st.write(data)
st.subheader("Random Tweets")
random_tweet = st.radio('Sentiment',('positive','neutral','negative'))
st.markdown(data.query('airline_sentiment==@random_tweet')[['text']].sample(n=1).iat[0,0])


st.sidebar.markdown("Visualizing the sentiments")
select = st.sidebar.selectbox('Type of Chart!',['histogram','pie-chart'], key=1)
sentiment_count = data['airline_sentiment'].value_counts()

sentiment_count = pd.DataFrame({'Sentiment':sentiment_count.index, 'Tweets':sentiment_count.values})
if not st.sidebar.checkbox("Hide", True):
    st.markdown("## Number of tweets by sentiment")
    if select == 'histogram':
        fig = px.bar(sentiment_count,x='Sentiment',y='Tweets', color='Tweets',height=500)
        st.plotly_chart(fig)

    else:
        fig = px.pie(sentiment_count,values='Tweets',names='Sentiment')
        st.plotly_chart(fig)
        
st.set_option('deprecation.showPyplotGlobalUse', False)

st.sidebar.subheader("Tweets by Airlines")
choice = st.sidebar.multiselect('Pick airlines',('US Airways','United','American','Southwest','Delta','Virgin America'), key=0)

if len(choice)>0:
    choice_data = data[data.airline.isin(choice)]
    fig_choice = px.histogram(choice_data, x='airline',y='airline_sentiment',histfunc='count',color='airline_sentiment',facet_col='airline_sentiment',labels={'airline_sentiment':'tweets'},height=800,width=600)
    st.plotly_chart(fig_choice)

st.sidebar.header("Word Cloud")
word_sentiment = st.sidebar.radio('Sentiment Word Cloud',('positive','neutral','negative'))
if not st.sidebar.checkbox("Close",True,key='3'):
    st.subheader("Word Cloud for %s sentiment"%(word_sentiment))
    df = data[data['airline_sentiment']==word_sentiment]
    words = ' '.join(df['text'])
    pro_words = ' '.join([word for word in words.split() if 'http' not in word and not word.startswith('@') and word!= 'RT'])
    wordcloud = WordCloud(stopwords=STOPWORDS, background_color='white',height=640,width=640).generate(pro_words)
    plt.imshow(wordcloud)
    plt.xticks([])
    plt.yticks([])
    st.pyplot()

import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
def predict(text):
  modelP = tensorflow.keras.models.load_model("tweetModel.h5")
  tok = Tokenizer(num_words =10000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
  tok.fit_on_texts(data.text.values)
  sequences = tok.texts_to_sequences(data.text.values)
  word_index = tok.word_index
  X_data = pad_sequences(sequences, maxlen = 100)
  t = [text]
  seq = tok.texts_to_sequences(t)
  padded = pad_sequences(seq, maxlen=100)
  pred = modelP.predict(padded)
  labels = ['positive', 'negative','neutral']
  label = labels[np.argmax(pred)]
  percent = max(pred) * 100
  st.write('Predicted Sentiment: '+label)
  st.write('Confident:'+str(percent))

st.sidebar.header("Check out new tweets!😎")
if not st.sidebar.checkbox("Hide", True, key=10):
  text = st.text_input('Tweet', 'Airline sentiment tweets')
  st.button('Predict Sentiment',on_click=predict(text))



Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501